In [ ]:
from reservoir import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Load sample data

In [ ]:
data = np.loadtxt('example_data/MackeyGlass_t17.txt')

train = data[:4000].reshape(-1, 1)
test = data[4000:6000].reshape(-1, 1)
plt.title("Train data")
plt.plot(train)
plt.show()
plt.title("Zoom in on first 400 train points")
plt.plot(train[:400])
plt.show()

### Add noise

In [ ]:
# Define noisy data for Bayesian Optimization
snr = 4
noise_std = np.sqrt(train.std(ddof=1)**2 / snr)

print('Standard deviation of noise is:', noise_std)
noisy_train = train + np.random.normal(0, noise_std, size=train.shape)
noisy_test = test + np.random.normal(0, noise_std, size=test.shape)
plt.plot(noisy_train[:400])
plt.show()

### Combine multiple series for evaluation if desired

In [ ]:
multiple_train = np.hstack((train, noisy_train))
multiple_test = np.hstack((test, noisy_test))

### Set bounds

In [ ]:
bounds = {
    'input_scaling': (0, 1),
    'feedback_scaling': (0, 1),
    'leaking_rate' : (0, 1), 
    'spectral_radius': (0, 1.25),
    'regularization': (-12, 1),
    'connectivity': (-3, 0),
    'n_nodes':  (100, 1000)
}

### Optimize

In [ ]:
# Set optimization parameters
esn_cv = EchoStateNetworkCV(bounds=bounds,
                            initial_samples=100,
                            subsequence_length=250,
                            eps=1e-3,
                            cv_samples=3, 
                            max_iterations=1000, 
                            scoring_method='tanh',
                            verbose=True)

In [ ]:
# Optimize (this may take a while!)
best_arguments = esn_cv.optimize(y=multiple_train)

In [ ]:
# Train best model(s)
esn = EchoStateNetwork(**best_arguments)
esn.train(y=train)

esn2 = EchoStateNetwork(**best_arguments)
esn2.train(y=noisy_train)

In [ ]:
# Test
score = esn.test(y=test, scoring_method='rmse')
score2 = esn.test(y=noisy_test, scoring_method='rmse')
print(score)
print(score2)

### Inspect performance

In [ ]:
# Diagnostic plots
plt.plot(esn.predict(100), label='Predicted')
plt.plot(test[:100], label='Ground truth')
plt.title('Prediction on next 100 steps')
plt.legend()
plt.show()

plt.plot(esn2.predict(100), label='Predicted')
plt.plot(noisy_test[:100], label='Ground truth')
plt.title('Prediction on next 100 steps')
plt.legend()
plt.show()

## Manually tuned example, for comparison

In [ ]:
# Regular approach
esn = EchoStateNetwork(n_nodes=1000, connectivity=0.01, input_scaling=0.5, feedback_scaling=0.5, leaking_rate=0.3, 
                       spectral_radius=1.25, regularization=1e-8, feedback=True)
complete_data, y_used, burn_in = esn.train(y=train, burn_in=100)

In [ ]:
# Prediction sample for ground truth
plt.plot(test[:500], label='Test')
plt.plot(esn.predict(500), label='Predicted')
plt.legend()
esn.test(y=test[:500], scoring_method='nrmse')

In [ ]:
# Stepwise prediction
five_step_prediction = esn.predict_stepwise(test[:20], steps_ahead=5)
plt.plot(test[:20], label='Test')
plt.plot(five_step_prediction, label='Predicted')
plt.legend();

In [ ]:
five_step_prediction